# Cyclistic Bike-Share Analysis

Author: Muh Ridwan Sukri
Last updated: January, 17th 2023

## Introduction
This is a fictional case study as a capstone project of the __[Google Data Analytics Certificate](https://grow.google/certificates/data-analytics/)__
My main goal is to show both my technical skills in SQL, R and Tableau as well as my data analyst approach to a case.
The whole process could have been done entirely in Python without SQL or another BI Tools.
In this case study, the six data analysis proces: ask, prepare, process, analyze, share, and act will be followed.

## Ask

### Background
I am a junior data analyst working in the marketing analyst team at Cyclistic, a bike-share company in Chicago that features more than 5,800 bicycles and 600 docking stations. The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. Therefore, my team wants to understand how casual riders and annual members use Cyclistic bikes differently. From these insights, my team will design a new marketing strategy to convert casual riders into annual members. But first, Cyclistic executives must approve my recommendations, so they must be backed up with compelling data insights and professional data visualizations.

### Business Tasks
1. Analyze Cyclistic historical bike trip data to identify trends and get some insights.
2. Provide high quality recommendations for marketing analyst team.

### Business Objectives
1. How do annual members and casual riders use Cyclistic bikes differently?
2. Why would casual riders buy Cyclistic annual memberships?
3. How can Cyclistic use digital media to influence casual riders to become members?

### Deliverables
1. A clear summary of the business task
2. A description of all data sources used
3. Documentation of any cleaning or manipulation of data
4. A summary of analysis
5. Supporting visualizations and key findings
6. High-level content recommendations based on the analysis


### Key stakeholders
- Lily Moreno: The director of marketing and your manager.
- Cyclistic marketing analytics team: A team of data analysts who are responsible for collecting, analyzing, and reporting data that helps guide Cyclistic marketing strategy.
- Cyclistic executive team: The notoriously detail-oriented executive team will decide whether to approve the recommended marketing program.

## Prepare

In the Prepare phase, we identify the data being used and its limitations.


### Information on Data Source
1. Cylistic dataset has been made available by Lyft Bikes and Scooters, LLC under this __[license](https://ride.divvybikes.com/data-license-agreement)__ and can be public accessed __[here](https://divvy-tripdata.s3.amazonaws.com/index.html)__
2. We are using another dataset to get hourly average of Temperature, Precipitation, Relative Humidity, Wind Speed, and many variables from [Visual Crossing](https://www.visualcrossing.com/weather-data).


### Crebility of Data
The credibility and integrity of our data can be determined using the ROCCC system.
- The data is __reliable__ — The dataset has a large sample size, reflecting the population size.
- The data is __original__ — The dataset is generated by the first source (Lyft Bikes and Scooters, LLC) and not from Third Party.
- The data is __comprehensive__ — There is no information about the participants, such as gender, age, health state, etc. This could mean that data was not randomised. If the data is biased, then the insights from the analysis will be unfair to all types of people. But it's understandable because this is public data so personal information must be hidden. Only an anonymous _ride_id_ identifies the ride.
- The data is __current__ — it is relevant and up to date, thus indicating that the source refreshes its data regularly on each month.
- The data is __cited__ — the source has been vetted.

### Tools
For this project, we are using some tools like:
- Some tools from Google Cloud Platform for ETL process;
- Python for data cleansing, EDA, and analysis;
- Tableau for creating dashboard.

## Process
We are using tools from GCS and Python to prepare and process the data.

### ETL Process
This is ETL structure that we used:
![ETL Process](https://archive.org/download/etl-process/etl-process.png)

The ETL process starts by downloading all files from the AWS bucket ["divvy-tripdata"](https://divvy-tripdata.s3.amazonaws.com/index.html), then we unzip all files and csv files from unzip is mounted to Google Cloud Storage using [GCSFuse](https://github.com/GoogleCloudPlatform/gcsfuse). Because the maximum uploaded file size from computer allowed by Bigquery is 100MB, so files must first be moved to the Cloud Storage Bucket and then uploaded to Bigquery.

After uploading all the files to Bigquery, we perform the following ["query"](https://github.com/ridwansukri/cyclistic-bike-share-analysis/blob/main/bigquery-etl.sql). And then, we join the main dataset with dataframe (we uploaded in Bigquery) from NASA that contains average temperature, Temperature, Precipitation, Relative Humidity, and Wind Speed with this ["query"](https://github.com/ridwansukri/cyclistic-bike-share-analysis/blob/main/join-with-nasa.sql).

### Metadata
This dataset contains some variables that we used to perform analysis. Some variables are original from the dataset and the others are the results of the transformation from original variable.

- ride_id: ID attached to each trip taken;
- rideable_type: Type of bicycle used by the customer;
- started_at: day and time trip started, in CST;
- ended_at: day and time trip ended, in CST;
- start_station_name: name of station where trip originated;
- start_station_id: ID of station where trip originated;
- end_station_name: name of station where trip terminated;
- end_station_id: ID of station where trip terminated
- start_lat: station latitude where trip originated;
- start_lng: station longitude where trip originated;
- end_lat: station latitude where trip terminated;
- start_lng: station longitude where trip terminated;
- member_casual: Type of customer that take the trip, casual or member;
- trip_duration: time of trip in seconds;
- YEAR: year when the trip started;
- MO : Month when the trip started;
- DY : Date when the trip started;
- HR : Hour when the trip started;
- day_start: the day when the trip started, starts from 1(Sunday) to 7 (Saturday).


### Import Libraries
Packages that we used are installed and aliased for easy reading.

In [1]:
# Import the necessary libraries
from google.cloud import bigquery #get data from Bigquery
import os # interacting with our OS
from google.cloud.exceptions import NotFound
from sqlalchemy import create_engine
import pandas as pd # data structure and data analysis
import numpy as np # data arrays
import matplotlib.pyplot as plt # data visualization library
import plotly.express as px #data visualization library
import seaborn as sns #data visualization library
sns.set_theme(style="whitegrid")

### Importing dataset
We import dataset from Bigquery and put in our notebook.

In [2]:
# Set environment variables
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/aspir/Desktop/Divvy/acoustic-portal-322707-496a5c838490.json'
# initiates BQ service
bigquery_client = bigquery.Client()
# Write Query on BQ
QUERY = """
SELECT *
FROM
  `bike_dataset.bike_trip`
;
  """
# Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()

### Data Cleaning and Manipulation
Steps

- Observe and familiarize with data;
- Check for null or missing values;
- Perform sanity check of data

Using head function to show first 10 rows and familiarise with the data. We create copy of dataframe as a checkpoint so if there is a wrong step that changes the dataframe to be unwanted, we don't need to run the syntax from the beginning, just run it from the checkpoint.

In [3]:
# View top few rows of result
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,YEAR,MO,DY,HR,day_start
0,B06B3C2BCB553751,classic_bike,2022-07-09 13:03:48+00:00,2022-07-09 13:09:26+00:00,Wells St & Walton St,TA1306000011,Larrabee St & Kingsbury St,TA1306000009,41.899930,-87.634430,41.897764,-87.642884,casual,338,2022,7,9,13,7
1,0EEF17FE68115937,classic_bike,2022-12-03 12:00:03+00:00,2022-12-03 12:07:47+00:00,Halsted St & Clybourn Ave,331,Larrabee St & Kingsbury St,TA1306000009,41.909668,-87.648128,41.897764,-87.642884,member,464,2022,12,3,12,7
2,CA5C3C61BD9D0763,classic_bike,2022-10-08 05:30:21+00:00,2022-10-08 05:37:11+00:00,Dayton St & North Ave,13058,Larrabee St & Kingsbury St,TA1306000009,41.910578,-87.649422,41.897764,-87.642884,casual,410,2022,10,8,5,7
3,54280A6BA04278FA,electric_bike,2022-05-28 11:16:56+00:00,2022-05-28 11:38:44+00:00,LaSalle Dr & Huron St,KP1705001026,Larrabee St & Kingsbury St,TA1306000009,41.894739,-87.632186,41.897764,-87.642884,casual,1308,2022,5,28,11,7
4,39B0C56254EE1E36,classic_bike,2022-01-22 19:56:32+00:00,2022-01-22 20:05:57+00:00,Eckhart Park,13289,Orleans St & Elm St,TA1306000006,41.896373,-87.660984,41.902924,-87.637715,member,565,2022,1,22,19,7


In [51]:
# Create a copy of dataframe
bike_df = df.copy()

Then, we check all column names, column types, and total rows.

In [52]:
# Checking all column types
print(bike_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5667907 entries, 0 to 5667906
Data columns (total 19 columns):
 #   Column              Dtype              
---  ------              -----              
 0   ride_id             object             
 1   rideable_type       object             
 2   started_at          datetime64[ns, UTC]
 3   ended_at            datetime64[ns, UTC]
 4   start_station_name  object             
 5   start_station_id    object             
 6   end_station_name    object             
 7   end_station_id      object             
 8   start_lat           float64            
 9   start_lng           float64            
 10  end_lat             float64            
 11  end_lng             float64            
 12  member_casual       object             
 13  trip_duration       int64              
 14  YEAR                int64              
 15  MO                  int64              
 16  DY                  int64              
 17  HR                  int64  

There are several column types that need to be changed.
Lily Moreno asked to retrieve data in the last 12 months. We will check the data frame.

In [53]:
# Check time ranges of bike_df
date_min_max = bike_df['started_at'].agg(['min', 'max'])
print(date_min_max)

min   2021-12-31 18:00:21+00:00
max   2022-12-31 17:59:57+00:00
Name: started_at, dtype: datetime64[ns, UTC]


Dataframe contains the last 12 months.
Then, find out whether there is any null or missing values in `bike_df` dataframe

In [54]:
# Check percentage of null values
print(bike_df.isnull().mean() * 100)

ride_id                0.000000
rideable_type          0.000000
started_at             0.000000
ended_at               0.000000
start_station_name    14.700153
start_station_id      14.700153
end_station_name      15.753293
end_station_id        15.753293
start_lat              0.000000
start_lng              0.000000
end_lat                0.103319
end_lng                0.103319
member_casual          0.000000
trip_duration          0.000000
YEAR                   0.000000
MO                     0.000000
DY                     0.000000
HR                     0.000000
day_start              0.000000
dtype: float64


We found some columns had missing value. We will mark to the next manipulation process.
Then we check whether there are duplicate rows in `bike_df` dataframe by checking the `ride_id` column

In [55]:
# Get duplicate rows
duplicates = bike_df.duplicated(subset=['ride_id'])
bike_df[duplicates]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,YEAR,MO,DY,HR,day_start


From above observation, we notice that:
1. There is no duplicate rows
2. Dataframe has 5731898 rows and 23 columns
3. `rideable_type`, `start_station_name`, `start_station_id`, `end_station_name`, `end_station_id`, `member_casual`, `year`, `month`, `date`, `hour`, `day_start` columns type need to be converted to `category` dtype
4. Some columns need to rename
5. Some columns associated with stations have null values with range of 14-16%
6. `end_lat` and `end_lng` have 1% of null values

Following data manipulation is performed:
- Rename some column names

In [56]:
# Rename columns
bike_df = bike_df.rename(columns={'rideable_type':'bike_type', 'member_casual':'customer_type', 'MO':'month',
                                  'DY':'date', 'YEAR':'year', 'HR':'hour'})

- Convert column types to right type.

In [57]:
# Define a category column converter function
def convert_to_category(df, columns):
    for column in columns:
        df[column] = df[column].astype('category')
    return df

# Define a string column converter function
def convert_to_string(df, columns):
    for column in columns:
        df[column] = df[column].astype('string')
    return df

# Define a int64 column converter function
def convert_to_int(df, columns):
    for column in columns:
        df[column] = df[column].astype(int)
    return df

In [58]:
# call converter column type function to category type
bike_df = convert_to_category(bike_df, ['bike_type', 'customer_type', 'day_start', 'year', 'month', 'date', 'hour'])

In [59]:
# Checking again data types
print(bike_df.dtypes)

ride_id                            object
bike_type                        category
started_at            datetime64[ns, UTC]
ended_at              datetime64[ns, UTC]
start_station_name                 object
start_station_id                   object
end_station_name                   object
end_station_id                     object
start_lat                         float64
start_lng                         float64
end_lat                           float64
end_lng                           float64
customer_type                    category
trip_duration                       int64
year                             category
month                            category
date                             category
hour                             category
day_start                        category
dtype: object


- Checking unique values using simple function. We noticed "docked_bike" just has 3% of total values in `bike_type` column. As our judgement, "docked_bike "is new version of "classic bike" which "docked_bike" has a portable lock so if you want to secure the bike it can be locked anywhere (no need to bring it to the near station)

In [60]:
# Checking unique values
for col in ['bike_type','customer_type', 'day_start', 'year', 'month', 'date', 'hour']:
    print(bike_df[col].unique())

['classic_bike', 'electric_bike', 'docked_bike']
Categories (3, object): ['classic_bike', 'docked_bike', 'electric_bike']
['casual', 'member']
Categories (2, object): ['casual', 'member']
[7, 6, 1, 2, 3, 4, 5]
Categories (7, int64): [1, 2, 3, 4, 5, 6, 7]
[2022, 2021]
Categories (2, int64): [2021, 2022]
[7, 12, 10, 5, 1, ..., 11, 3, 9, 6, 2]
Length: 12
Categories (12, int64): [1, 2, 3, 4, ..., 9, 10, 11, 12]
[9, 3, 8, 28, 22, ..., 4, 21, 7, 12, 26]
Length: 31
Categories (31, int64): [1, 2, 3, 4, ..., 28, 29, 30, 31]
[13, 12, 5, 11, 19, ..., 2, 3, 6, 7, 4]
Length: 24
Categories (24, int64): [0, 1, 2, 3, ..., 20, 21, 22, 23]


In [61]:
# Checking percentage of each value on bike_type column
print(bike_df['bike_type'].value_counts(normalize=True))

electric_bike    0.509746
classic_bike     0.458939
docked_bike      0.031315
Name: bike_type, dtype: float64


In [62]:
# Change 'docked_bike' to 'classic_bike'
bike_df['bike_type'] = bike_df['bike_type'].replace(['docked_bike'],'classic_bike')
print('bike type: ', bike_df['bike_type'].unique())

bike type:  ['classic_bike', 'electric_bike']
Categories (2, object): ['classic_bike', 'electric_bike']


- Transform all values in "bike_type" and "customer_type" as title case.

In [63]:
# Convert values in bike_type as title case and remove underscore
bike_df['bike_type'] = bike_df['bike_type'].str.title().replace('_', ' ', regex=True)
print(bike_df['bike_type'].value_counts())

Electric Bike    2889191
Classic Bike     2778716
Name: bike_type, dtype: int64


In [64]:
# Convert values in customer_type as title case and remove underscore
bike_df['customer_type'] = bike_df.customer_type.str.title()
print(bike_df['customer_type'].value_counts())

Member    3345771
Casual    2322136
Name: customer_type, dtype: int64


- `start_station_name` and `end_station_name` end with 1.3% unusual symbols , such as *, (TEMP), and TEST symbols. We need to remove it.

In [65]:
# Create variable with unusual symbols
search = ('*', '(Temp)', 'TEST')

In [66]:
# Checking start_station_name and end_station_name whether have unusual symbols
print(bike_df["start_station_name"].str.endswith(search).value_counts(normalize=True))
print(bike_df["end_station_name"].str.endswith(search).value_counts(normalize=True))

False    0.984342
True     0.015658
Name: start_station_name, dtype: float64
False    0.983615
True     0.016385
Name: end_station_name, dtype: float64


In [67]:
# Remove unusual symbols
bike_df['start_station_name'] = bike_df['start_station_name'].str.rstrip('(*) (Temp) (TEST)')
bike_df['end_station_name'] = bike_df['end_station_name'].str.rstrip('(*) (Temp) (TEST)')

In [68]:
# Checking again both columns if still contains unusual symbols
print(bike_df["start_station_name"].str.endswith(search).value_counts(normalize=True))
print(bike_df["end_station_name"].str.endswith(search).value_counts(normalize=True))

False    1.0
Name: start_station_name, dtype: float64
False    1.0
Name: end_station_name, dtype: float64


- We need to change all value of `day_start` column to make it easier to read. Value 1 is represented by "Sunday" and 7 by "Saturday" in a row.

In [69]:
# Create mapping dictionary and replace
mapping = {1:'Sunday', 2:'Monday', 3:'Tuesday', 4:'Wednesday', 5:'Thursday', 6:'Friday', 7:'Saturday'}
bike_df['day_start'] = bike_df['day_start'].replace(mapping).astype('category')

In [70]:
# Check the replaced values
print(bike_df['day_start'].value_counts())

Saturday     915226
Thursday     841591
Friday       803222
Wednesday    798223
Tuesday      782372
Sunday       776259
Monday       751014
Name: day_start, dtype: int64


- Remove UTC offset in `started_at` and `ended_at` columns.

In [71]:
# Remove UTC offset in started_at and ended_at column
bike_df['started_at'] = pd.to_datetime(bike_df['started_at']).dt.tz_convert(None)
bike_df['ended_at'] = pd.to_datetime(bike_df['ended_at']).dt.tz_convert(None)

In [72]:
# Checking after remove UTC offset
print(bike_df[['started_at', 'ended_at']].head())

           started_at            ended_at
0 2022-07-09 13:03:48 2022-07-09 13:09:26
1 2022-12-03 12:00:03 2022-12-03 12:07:47
2 2022-10-08 05:30:21 2022-10-08 05:37:11
3 2022-05-28 11:16:56 2022-05-28 11:38:44
4 2022-01-22 19:56:32 2022-01-22 20:05:57


- We remove any trips that were below 60 seconds in length (potentially false starts or users trying to re-dock a bike to ensure it was secure). And From [this](https://gov.publicstuff.com/content/kb/529/view/52885) website, if the bike has not been returned and correctly docked at a station after 24 hours, the bike is considered stolen and we need to remove from our data because this ride is invalid.

In [73]:
# Drop rides when the rides less than 60 seconds and more than 24 hours
bike_df = bike_df[(bike_df['trip_duration'] > 60) & (bike_df['trip_duration'] < 86400)]

# Check again
duration_min_max = bike_df['trip_duration'].agg(['min', 'max'])
print(duration_min_max)

min       61
max    86396
Name: trip_duration, dtype: int64


- We change the `trip_duration` column from seconds to minutes for easy to read.

In [74]:
#convert trip_duration from seconds to minutes
bike_df['trip_duration'] = bike_df['trip_duration'].div(60)
# Check after convert
print(bike_df['trip_duration'].head())

0     5.633333
1     7.733333
2     6.833333
3    21.800000
4     9.416667
Name: trip_duration, dtype: float64


- We drop end_lat column which had null values. Our assumption that ride is invalid and maybe tke bike get stolen, if the app can get the starting coordinates, it should also be able to get the ending coordinates.

In [75]:
# Drop missing values in end_lat and end_lng columns
bike_df = bike_df.dropna(subset = ['end_lat', 'end_lng'])
# Check percentage of null values
print(bike_df.isnull().mean() * 100)

ride_id                0.000000
bike_type              0.000000
started_at             0.000000
ended_at               0.000000
start_station_name    14.475610
start_station_id      14.475610
end_station_name      15.241805
end_station_id        15.241805
start_lat              0.000000
start_lng              0.000000
end_lat                0.000000
end_lng                0.000000
customer_type          0.000000
trip_duration          0.000000
year                   0.000000
month                  0.000000
date                   0.000000
hour                   0.000000
day_start              0.000000
dtype: float64


- The rest of the columns that still have null values will be filled with "Not Available". We're assuming the bike starts or ends at a non-station, or the app has trouble getting the station name.

In [76]:
# Fill null values with Not Available
bike_df[['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']] = bike_df[['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']].fillna("Not Available")

In [77]:
# Check percentage of null values
print(bike_df.isnull().mean() * 100)

ride_id               0.0
bike_type             0.0
started_at            0.0
ended_at              0.0
start_station_name    0.0
start_station_id      0.0
end_station_name      0.0
end_station_id        0.0
start_lat             0.0
start_lng             0.0
end_lat               0.0
end_lng               0.0
customer_type         0.0
trip_duration         0.0
year                  0.0
month                 0.0
date                  0.0
hour                  0.0
day_start             0.0
dtype: float64


- Create `season` column based on `started_at` column.

In [78]:
# create date_offset column
bike_df['date_offset'] = (bike_df['started_at'].dt.month*100 + bike_df['started_at'].dt.day - 320)%1300

#Categorizing the season based on date_offset
bike_df['season'] = pd.cut(bike_df['date_offset'], [0, 300, 602, 900, 1300],
                      labels=['Spring', 'Summer', 'Fall', 'Winter'])

# call our function to convert as category type
bike_df = convert_to_category(bike_df, ['season'])


In [79]:
# Checking Season value that created with above syntax
print(bike_df['season'].unique())

['Summer', 'Fall', 'Spring', 'Winter', NaN]
Categories (4, object): ['Spring' < 'Summer' < 'Fall' < 'Winter']


In [80]:
# Investigate season column that contains missing value (NaN)
nan_in_col = bike_df[bike_df['season'].isna()]
# Check range of started ride in season's missing value
date_min_max = nan_in_col['started_at'].agg(['min', 'max'])
print(date_min_max)

min   2022-03-20 00:00:04
max   2022-03-20 23:59:59
Name: started_at, dtype: datetime64[ns]


In [81]:
# 20 March in Chicago is spring, fill missing values with spring
bike_df['season'] = bike_df['season'].fillna("Spring")

In [82]:
# Check again unique value of season column
print(bike_df['season'].unique())

['Summer', 'Fall', 'Spring', 'Winter']
Categories (4, object): ['Spring' < 'Summer' < 'Fall' < 'Winter']


In [83]:
# Drop date_offset column that we don't use anymore
bike_df = bike_df.drop('date_offset', axis=1)

- Then, we change values in `month` from number to month name;

In [84]:
# Convert month number
mapping = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June',
           7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}

- Creating new `hour` column based on `started_at` column

In [85]:
# Get hour based on started_at column
bike_df['hour'] = bike_df.started_at.apply(lambda x: x.hour)

- In this final cleaning step, we need to change the imprecise column types

In [86]:
# Check again data types
print(bike_df.dtypes)

ride_id                       object
bike_type                     object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
customer_type                 object
trip_duration                float64
year                        category
month                       category
date                        category
hour                           int64
day_start                   category
season                      category
dtype: object


In [87]:
# call function to convert column as category type
bike_df = convert_to_category(bike_df, ['bike_type', 'start_station_name', 'start_station_id',
                                        'end_station_name', 'end_station_id', 'customer_type', 'month', 'hour'])

- In analyze step, we are using final dataframe `clean_bike`. We drop columns that we will not use in the analysis process;

In [88]:
# Drop column and creating new dataframe
clean_bike = bike_df.drop(columns=['ride_id', 'start_station_id', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',])

### Combining Main Dataframe with Weather Data
After we clean main dataframe, we will combine with weather data that we cleaned [here](https://app.datacamp.com/workspace/w/750fac7a-4b4a-45e8-9a60-8996b4a46dfd).

In [89]:
# Set environment variables
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/aspir/Desktop/Divvy/acoustic-portal-322707-496a5c838490.json'
# initiates BQ service
bigquery_client = bigquery.Client()
# Write Query on BQ
QUERY = """
SELECT *
FROM
  `bike_dataset.clean_weather`
;
  """
# Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
weather_df = Query_Results.to_dataframe()

In [90]:
# Check weather data
weather_df.head()

,MO,DY,HR,temp,feelslike,dew,humidity,precip,precipprob,snow,snowdepth,windspeed,winddir,sealevelpressure,cloudcover,visibility,condition
0,1,1,0,5.0,0.8,2.5,84.14,0.0,0,0.0,0.0,22.3,12.0,1006.7,100.0,8.6,Cloudy
1,1,1,1,4.2,-0.7,2.2,86.96,0.0,0,0.0,0.0,26.3,15.0,1007.1,100.0,5.3,Cloudy
2,1,1,2,3.5,-0.4,1.5,86.89,0.0,0,0.0,0.0,16.7,5.0,1007.9,100.0,5.3,Cloudy
3,1,1,3,3.6,-0.7,1.3,85.10,0.0,0,0.0,0.0,19.9,11.0,1008.5,100.0,9.1,Cloudy
4,1,1,4,3.5,-1.1,0.8,82.72,0.0,0,0.0,0.0,22.6,10.0,1008.4,100.0,13.6,Cloudy


In [91]:
# Rename Weather column name
weather_df = weather_df.rename(columns={'MO':'month', 'DY':'date', 'HR':'hour'})

In [92]:
# Left join clean_bike with weather dataframe
clean_bike = pd.merge(clean_bike, weather_df, on=['month', 'date', 'hour'])

In [93]:
clean_bike.head()

,bike_type,started_at,ended_at,start_station_name,end_station_name,customer_type,trip_duration,year,month,date,...,precip,precipprob,snow,snowdepth,windspeed,winddir,sealevelpressure,cloudcover,visibility,condition
0,Classic Bike,2022-07-09 13:03:48,2022-07-09 13:09:26,Wells St & Walton St,Larrabee St & Kingsbury St,Casual,5.633333,2022,7,9,...,0.0,0,0.0,0.0,22.1,64.0,1021.4,24.2,16.0,Partly Cloudy Day
1,Classic Bike,2022-07-09 13:35:40,2022-07-09 14:12:38,DuSable Lake Shore Dr & North Blvd,Michigan Ave & 8th St,Casual,36.966667,2022,7,9,...,0.0,0,0.0,0.0,22.1,64.0,1021.4,24.2,16.0,Partly Cloudy Day
2,Classic Bike,2022-07-09 13:28:48,2022-07-09 13:41:07,Central Park Ave & Elbridge Av,Avondale Ave & Irving Park Rd,Casual,12.316667,2022,7,9,...,0.0,0,0.0,0.0,22.1,64.0,1021.4,24.2,16.0,Partly Cloudy Day
3,Classic Bike,2022-07-09 13:31:14,2022-07-09 14:21:48,Lake Park Ave & 56th St,Michigan Ave & 8th St,Casual,50.566667,2022,7,9,...,0.0,0,0.0,0.0,22.1,64.0,1021.4,24.2,16.0,Partly Cloudy Day
4,Classic Bike,2022-07-09 13:30:42,2022-07-09 13:40:39,Wabash Ave & Roosevelt Rd,Shedd Aquariu,Casual,9.950000,2022,7,9,...,0.0,0,0.0,0.0,22.1,64.0,1021.4,24.2,16.0,Partly Cloudy Day


In [95]:
# Check percentage of null values
print(clean_bike.isnull().mean() * 100)

bike_type             0.0
started_at            0.0
ended_at              0.0
start_station_name    0.0
end_station_name      0.0
customer_type         0.0
trip_duration         0.0
year                  0.0
month                 0.0
date                  0.0
hour                  0.0
day_start             0.0
season                0.0
temp                  0.0
feelslike             0.0
dew                   0.0
humidity              0.0
precip                0.0
precipprob            0.0
snow                  0.0
snowdepth             0.0
windspeed             0.0
winddir               0.0
sealevelpressure      0.0
cloudcover            0.0
visibility            0.0
condition             0.0
dtype: float64


## Analysis
The analysis phase is still in progress...